In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [3]:
df =  pd.read_csv('data/sec__edgar_company_info.csv')
df.head()

,Line Number,Company Name,Company CIK Key
0,1,!J INC,1438823
1,2,"#1 A LIFESAFER HOLDINGS, INC.",1509607
2,3,#1 ARIZONA DISCOUNT PROPERTIES LLC,1457512
3,4,#1 PAINTBALL CORP,1433777
4,5,$ LLC,1427189


In [4]:
df.shape

(663000, 3)

In [15]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [17]:
print('All 3-grams in "Paintball":')
ngrams('Paintball')

All 3-grams in "Paintball":


['Pai', 'ain', 'int', 'ntb', 'tba', 'bal', 'all']